In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)

In [29]:
# Reading the log file
df = pd.read_csv('lang-error-17-12-2020.csv',index_col=False)

In [30]:
# Seeing data
df.head()

,IP,TIME,PLATFORM,REQ_URL,REQ_FEED,DESC
0,172.29.63.76,17-12-2020 0:0:5,desktop,https://eisamay.indiatimes.com/entertainment/c...,https://eisamay.indiatimes.com/pwafeeds/sc_gad...,Error: Request failed with status code 404
1,172.29.63.76,17-12-2020 0:0:7,desktop,https://eisamay.indiatimes.com/entertainment/c...,NaN,'Error [ERR_HTTP_HEADERS_SENT]: Cannot set hea...
2,172.29.63.76,17-12-2020 0:0:8,mobile,https://eisamay.indiatimes.com/west-bengal-new...,https://eisamay.indiatimes.com/pwafeeds/sc_gad...,Error: Request failed with status code 404
3,172.29.63.76,17-12-2020 0:0:14,mobile,https://eisamay.indiatimes.com/bangla-news-liv...,NaN,'TypeError: Cannot read property 'pagetype' of...
4,172.29.63.76,17-12-2020 0:0:16,desktop,https://eisamay.indiatimes.com/videos/entertai...,NaN,'TypeError: Cannot read property 'components' ...


In [31]:
# Seeing platform specific
# 71 percent of errors are in desktop in this ip while 28 in mobile
df['PLATFORM'].value_counts(normalize=True)

desktop    0.712873
mobile     0.287127
Name: PLATFORM, dtype: float64

In [32]:
## Seeing feed errors vs others
df[~(df['REQ_FEED'].isnull())].count()[0] / df.shape[0] 
# 41.8 percent feederrors 

0.41832697470679664

In [33]:
def categorize_url(url):
    templates = ['articleshow','photoshow','photolist','videoshow','articlelist','topics']
    for template in templates:
        if template in url:
            return template
    return 'others'
df['URL_CATEGORY'] = df['REQ_URL'].apply(categorize_url)


In [34]:
df['URL_CATEGORY'].value_counts()

articlelist    6028
articleshow    5244
videoshow      1511
photoshow       737
others          499
photolist       110
topics           25
Name: URL_CATEGORY, dtype: int64

In [56]:
# How i found the categories:
# First inspected the df, found 404 and undefined, started with these as a base along with null
# Characterized others as "others" manually inspected others one by one and
# found: 410 ,403, ['object Object'] timeout
def categorize_error(error):
    #     categories = ['null','undefined','404','410','ERR_HTTP_HEADERS_SENT','9000ms']
    categories = {
       '[object Object]':'[object Object]','undefined':'undefined','404':'404','410':'410','ECONNRESET':'ECONNRESET','ERR_HTTP_HEADERS_SENT':'ERR_HTTP_HEADERS_SENT','9000ms':'timeout'       
    }
    for cat in categories:
        if cat in error:
            return categories[cat]
    return 'other'
            
df['ERROR_TYPE'] = df['DESC'].apply(categorize_error)

In [57]:
df['ERROR_TYPE'].value_counts()

ERR_HTTP_HEADERS_SENT    6417
404                      5738
undefined                1713
410                       129
[object Object]           100
other                      27
timeout                    25
ECONNRESET                  5
Name: ERROR_TYPE, dtype: int64

In [58]:
# Very vague type of error
df['DESC'][df['ERROR_TYPE'] == '[object Object]']

107      '[object Object]'
112      '[object Object]'
119      '[object Object]'
125      '[object Object]'
181      '[object Object]'
336      '[object Object]'
663      '[object Object]'
700      '[object Object]'
932      '[object Object]'
933      '[object Object]'
946      '[object Object]'
979      '[object Object]'
1329     '[object Object]'
1332     '[object Object]'
1381     '[object Object]'
1385     '[object Object]'
1829     '[object Object]'
1901     '[object Object]'
2035     '[object Object]'
2039     '[object Object]'
2049     '[object Object]'
2050     '[object Object]'
2052     '[object Object]'
2053     '[object Object]'
2111     '[object Object]'
2171     '[object Object]'
2228     '[object Object]'
2422     '[object Object]'
2423     '[object Object]'
2425     '[object Object]'
               ...        
10467    '[object Object]'
10683    '[object Object]'
10685    '[object Object]'
10693    '[object Object]'
10720    '[object Object]'
10743    '[object Object]'
1

In [68]:
# Further inspection reveals that some urls are broken with '-' in between. It will be useful if we categorize these.
df['REQ_URL'][df['ERROR_TYPE'] == '[object Object]']

107      https://eisamay.indiatimes.com/nation/army-officer-shot-dead-by-jawan-in-uri-sector-in-jammu-and-kashmir/-/jawan/photoshow/59645394.cms                                                                                                                                                  
112      https://eisamay.indiatimes.com/business/business-news/jana-small-finance-bank-provides-9-6-interest-rate-on-fixed-deposit/photoshow/68437855.cms                                                                                                                                         
119      https://eisamay.indiatimes.com/lifestyle/health-fitness/6-proven-health-benefits-of-dates/photoshow/68343345.cms                                                                                                                                                                         
125      https://eisamay.indiatimes.com/business/business-news/jana-small-finance-bank-provides-9-6-interest-rate-on-fixed-depo

In [65]:
# Categories to check where we failed in urls
# Manually inspecting, i found '-/' '/-/' and cases like
df['IS_URL_ERROR'] = df['REQ_URL'].apply(lambda x: '/-/' in x or '-/' in x)

In [66]:
df['IS_URL_ERROR'].value_counts()

False    12643
True     1511 
Name: IS_URL_ERROR, dtype: int64